<a href="https://colab.research.google.com/github/ErdemAslans/AI-Powered-Live-Support-with-RAG-Django-Scalable-Intelligent-Assistance/blob/main/Multidisease_Anaylsis_Platform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!!pip install  segmentation_models_pytorch
!pip install monai
!pip install timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 5.7 MB/s eta 0:00:00


In [2]:
!pip install scikit-image

In [3]:
!pip install PyWavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 75.4 MB/s eta 0:00:00


In [4]:
!pip install bm3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 862.0/862.0 kB 45.3 MB/s eta 0:00:00


In [5]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 116.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import gradio as gr
import nibabel as nib
import numpy as np
import cv2
from skimage.transform import resize
from skimage import exposure
import pywt
from skimage.restoration import denoise_tv_chambolle
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm
import torchvision
from torchvision import models
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import logging

logging.basicConfig(level=logging.ERROR)

In [ ]:
class CBAMBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(CBAMBlock, self).__init__()
        self.channel_attention = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, channels // reduction, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(channels // reduction, channels, kernel_size=1),
            nn.Sigmoid()
        )
        self.spatial_attention = nn.Sequential(
            nn.Conv2d(2, 1, kernel_size=7, padding=3),
            nn.Sigmoid()
        )

    def forward(self, x):
        ca = self.channel_attention(x)
        x = x * ca
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        sa = torch.cat([avg_out, max_out], dim=1)
        sa = self.spatial_attention(sa)
        x = x * sa
        return x

class MultiScaleFeatureFusion(nn.Module):
    def __init__(self, embed_dim, num_classes=1, dropout_rate=0.1):
        super(MultiScaleFeatureFusion, self).__init__()
        self.conv1 = nn.Conv2d(embed_dim, embed_dim // 2, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(embed_dim, embed_dim // 2, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(embed_dim, embed_dim // 2, kernel_size=3, padding=1)
        self.conv_pred = nn.Conv2d(embed_dim // 2, num_classes, kernel_size=1)
        self.dropout = nn.Dropout(p=dropout_rate)

        nn.init.kaiming_normal_(self.conv1.weight)
        nn.init.kaiming_normal_(self.conv2.weight)
        nn.init.kaiming_normal_(self.conv3.weight)
        nn.init.kaiming_normal_(self.conv_pred.weight)

    def forward(self, x):
        x1 = F.interpolate(self.conv1(x), size=(x.shape[2], x.shape[3]), mode='bilinear', align_corners=False)
        x2 = self.conv2(x)
        x3 = F.interpolate(self.conv3(x), size=(x.shape[2], x.shape[3]), mode='bilinear', align_corners=False)
        fused = x1 + x2 + x3
        fused = self.dropout(fused)
        output = self.conv_pred(fused)
        return output

class ViTFeatureExtractor(nn.Module):
    def __init__(self, dropout_rate=0.2):
        super(ViTFeatureExtractor, self).__init__()
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True, drop_rate=dropout_rate, drop_path_rate=0.2)
        self.vit.head = nn.Identity()
        self.embed_dim = self.vit.embed_dim

    def forward(self, x):
        x = self.vit.forward_features(x)
        return x

class SegFormerDecoderWithAttention(nn.Module):
    def __init__(self, embed_dim, num_classes=1, patch_size=16, dropout_rate=0.4):
        super(SegFormerDecoderWithAttention, self).__init__()
        self.patch_size = patch_size
        self.num_patches_side = 224 // self.patch_size
        self.linear_fuse = nn.Linear(embed_dim, embed_dim)
        self.conv_pred = nn.Conv2d(embed_dim, num_classes, kernel_size=1)
        self.dropout = nn.Dropout(p=dropout_rate)
        self.cbam = CBAMBlock(embed_dim)
        self.multi_scale_fusion = MultiScaleFeatureFusion(embed_dim, num_classes, dropout_rate)
        nn.init.kaiming_normal_(self.linear_fuse.weight)
        nn.init.kaiming_normal_(self.conv_pred.weight)

    def forward(self, x):
        B, N, C = x.shape
        x = x[:, 1:, :]
        x = self.linear_fuse(x)
        x = x.transpose(1, 2).contiguous().view(B, C, self.num_patches_side, self.num_patches_side)
        x = F.interpolate(x, size=(224, 224), mode='bilinear', align_corners=False)
        x = self.cbam(x)
        x = self.multi_scale_fusion(x)
        return x

class ViT_SegFormer_Model(nn.Module):
    def __init__(self, dropout_rate=0.2):
        super(ViT_SegFormer_Model, self).__init__()
        self.vit = ViTFeatureExtractor(dropout_rate)
        self.decoder = SegFormerDecoderWithAttention(embed_dim=self.vit.embed_dim, dropout_rate=dropout_rate)

    def forward(self, x):
        features = self.vit(x)
        output = self.decoder(features)
        return output

In [ ]:
class CustomViT(nn.Module):
    def __init__(self, pretrained_vit, num_classes):
        super(CustomViT, self).__init__()
        self.vit = pretrained_vit
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(pretrained_vit.heads[0].in_features, num_classes)
        self.vit.heads[0] = self.fc

    def forward(self, x):
        x = self.vit(x)
        x = self.dropout(x)
        return x

In [ ]:
def enhance_contrast(image, clip_limit=0.6, tile_grid_size=(32,32), gamma=0.3):
    min_val = np.min(image)
    max_val = np.max(image)
    if max_val - min_val == 0:
        image = np.zeros_like(image)
    else:
        image = (image - min_val) / (max_val - min_val)

    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    image = clahe.apply((image * 255).astype(np.uint8)) / 255.0

    inv_gamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** inv_gamma * 255 for i in np.arange(0, 256)]).astype("uint8")
    image = cv2.LUT((image * 255).astype(np.uint8), table).astype(np.float32) / 255

    laplacian = cv2.Laplacian(image.astype(np.float64), cv2.CV_64F)
    laplacian = np.clip(laplacian, 0, 1)
    laplacian_weight = 0.33
    enhanced_image = image + laplacian_weight * laplacian
    enhanced_image = np.clip(enhanced_image, 0, 1)
    return enhanced_image

def apply_nlm_denoising(image):
    denoised_image = cv2.fastNlMeansDenoising(np.uint8(image * 255), None, h=10, templateWindowSize=7, searchWindowSize=21)
    denoised_image = denoised_image.astype(np.float32) / 255.0
    return denoised_image

def apply_wavelet_denoising(image):
    coeffs = pywt.wavedec2(image, 'haar', level=2)
    coeffs = list(coeffs)
    coeffs[1:] = [tuple(pywt.threshold(c, value=0.1, mode='soft') for c in level) for level in coeffs[1:]]
    denoised_image = pywt.waverec2(coeffs, 'haar')
    if denoised_image.shape != image.shape:
        denoised_image = resize(denoised_image, image.shape, mode='reflect', anti_aliasing=True)
    return denoised_image

def apply_total_variation_denoising(image):
    denoised_image = denoise_tv_chambolle(image, weight=0.1)
    return denoised_image

def process_single_nii(image_nii, mask_nii, view, slice_idx, target_size=(224,224)):
    img = image_nii.get_fdata()
    if mask_nii is not None:
        mask = mask_nii.get_fdata()
    else:
        mask = np.zeros_like(img)

    if view == 'sagittal':
        max_slices = img.shape[0]
        img_slice = img[slice_idx, :, :] if slice_idx < max_slices else img[max_slices//2,:,:]
        mask_slice = mask[slice_idx, :, :] if slice_idx < max_slices else mask[max_slices//2,:,:]
    elif view == 'coronal':
        max_slices = img.shape[1]
        img_slice = img[:, slice_idx, :] if slice_idx < max_slices else img[:, max_slices//2,:]
        mask_slice = mask[:, slice_idx, :] if slice_idx < max_slices else mask[:, max_slices//2,:]
    elif view == 'axial':
        max_slices = img.shape[2]
        img_slice = img[:, :, slice_idx] if slice_idx < max_slices else img[:,:,max_slices//2]
        mask_slice = mask[:, :, slice_idx] if slice_idx < max_slices else mask[:,:,max_slices//2]

    img_slice = resize(img_slice, target_size, mode='reflect', anti_aliasing=True)
    mask_slice = resize(
        mask_slice, target_size, mode='reflect', anti_aliasing=False,
        order=0, preserve_range=True
    )

    img_slice = enhance_contrast(img_slice)
    img_slice = apply_nlm_denoising(img_slice)
    img_slice = apply_wavelet_denoising(img_slice)
    img_slice = apply_total_variation_denoising(img_slice)

    mask_slice = (mask_slice > 0.05).astype(np.float32)

    highlight_factor = 1.8
    darken_factor = 0.8
    img_slice_masked = img_slice * mask_slice * highlight_factor
    img_slice_non_masked = img_slice * (1 - mask_slice) * darken_factor
    img_slice = img_slice_masked + img_slice_non_masked
    img_slice = np.clip(img_slice, 0, 1).astype(np.float32)

    return img_slice, mask_slice


In [ ]:

def get_albumentations_transform():
    transform = A.Compose([
        A.Resize(224, 224),
        A.HorizontalFlip(),
        A.RandomCrop(224, 224),
        A.CoarseDropout(max_holes=8, max_height=16, max_width=16, fill_value=0),
        A.RandomBrightnessContrast(p=0.2),
        A.ShiftScaleRotate(p=0.2),
        A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
        ToTensorV2()
    ])
    return transform

def process_pulmonary_image_specific(image, augmentation_pipeline):
    try:
        augmented = augmentation_pipeline(image=image)
        image_transformed = augmented['image']
        return image_transformed
    except Exception as e:
        logging.error(f"Pulmonary hypertension preprocessing error: {e}")
        return None



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

tumor_model = ViT_SegFormer_Model(dropout_rate=0.2).to(device)
tumor_weight_path = "/content/drive/MyDrive/Tumor/best_model.pth"
try:
    state_dict_tumor = torch.load(tumor_weight_path, map_location=device)
    tumor_model.load_state_dict(state_dict_tumor)
    tumor_model.eval()
    print("Breast Cancer Segmentation model loaded successfully.")
except Exception as e:
    print(f"Error loading Breast Cancer Segmentation model: {e}")

pretrained_vit = torchvision.models.vit_b_16(weights=torchvision.models.ViT_B_16_Weights.DEFAULT).to(device)

pulmonary_model = CustomViT(pretrained_vit, num_classes=4).to(device)
pulmonary_weight_path = "/content/drive/MyDrive/Pulmonary_Hypertension/best_model.pth"
try:
    state_dict_pulmonary = torch.load(pulmonary_weight_path, map_location=device)
    pulmonary_model.load_state_dict(state_dict_pulmonary)
    pulmonary_model.eval()
    print("Pulmonary Hypertension Analysis model loaded successfully.")
except Exception as e:
    print(f"Error loading Pulmonary Hypertension Analysis model: {e}")



In [ ]:
def process_tumor_image(image_file, mask_file, view, slice_idx):
    try:
        if not image_file:
            raise ValueError("Image file not uploaded.")

        image_nii = nib.load(image_file.name)
        mask_nii = nib.load(mask_file.name) if mask_file and mask_file.name else None

        img_slice, mask_slice = process_single_nii(image_nii, mask_nii, view, slice_idx, target_size=(224,224))

        img_tensor = torch.from_numpy(img_slice).unsqueeze(0).unsqueeze(0)
        img_tensor = img_tensor.repeat(1, 3, 1, 1)
        img_tensor = img_tensor.to(device)

        with torch.no_grad():
            logits = tumor_model(img_tensor)
            pred_prob = torch.sigmoid(logits)
            pred_mask = (pred_prob > 0.5).float()

        pred_mask_np = pred_mask.cpu().squeeze().numpy()

        return (img_slice, pred_mask_np)
    except FileNotFoundError as e:
        logging.error(f"File not found: {e}")
        return "File not found.", "File not found."
    except ValueError as e:
        logging.error(f"Value error: {e}")
        return "Invalid input.", "Invalid input."
    except Exception as e:
        logging.error(f"Unexpected error: {e}")
        return "An error occurred.", "An error occurred."

def process_pulmonary_image(image):
    try:
        if image is None:
            raise ValueError("Image not uploaded.")

        if isinstance(image, np.ndarray):
            image = Image.fromarray(image.astype('uint8'))

        augmentation_pipeline = get_albumentations_transform()

        image_transformed = process_pulmonary_image_specific(np.array(image), augmentation_pipeline)
        if image_transformed is None:
            raise ValueError("Pulmonary hypertension preprocessing failed.")

        img_tensor = image_transformed.unsqueeze(0).to(device)

        with torch.no_grad():
            outputs = pulmonary_model(img_tensor)
            probabilities = torch.softmax(outputs, dim=1)
            predicted_class = torch.argmax(probabilities, dim=1).item()
            confidence = probabilities[0][predicted_class].item()

        class_names = ['Normal', 'Mild PH', 'Moderate PH', 'Severe PH']
        prediction = class_names[predicted_class]

        return prediction, f"{confidence:.2%}"
    except ValueError as e:
        logging.error(f"Value error: {e}")
        return "Invalid input.", "0.00%"
    except Exception as e:
        logging.error(f"Unexpected error: {e}")
        return "An error occurred.", "0.00%"

with gr.Blocks() as demo:
    gr.Markdown("# Medical Image Analysis System")

    with gr.Tabs():
        with gr.Tab("Breast Cancer Segmentation"):
            with gr.Row():
                image_input = gr.File(label="NIfTI Image File (.nii or .nii.gz)", file_count="single")
                mask_input = gr.File(label="Mask File (Optional)", file_count="single")
            view_choice = gr.Dropdown(
                choices=["sagittal", "coronal", "axial"],
                value="axial",
                label="View Axis"
            )
            slice_choice = gr.Slider(
                minimum=0,
                maximum=200,
                step=1,
                value=50,
                label="Slice Number"
            )
            tumor_button = gr.Button("Perform Segmentation")
            output_image = gr.Image(label="Processed Image")
            output_mask = gr.Image(label="Segmentation Mask")

        with gr.Tab("Pulmonary Hypertension Analysis"):
            ph_image_input = gr.Image(label="Tomography Image")
            ph_button = gr.Button("Perform PH Analysis")
            with gr.Row():
                ph_prediction = gr.Label(label="Prediction")
                ph_confidence = gr.Label(label="Confidence Score")

    tumor_button.click(
        fn=process_tumor_image,
        inputs=[image_input, mask_input, view_choice, slice_choice],
        outputs=[output_image, output_mask]
    )

    ph_button.click(
        fn=process_pulmonary_image,
        inputs=ph_image_input,
        outputs=[ph_prediction, ph_confidence]
    )

demo.launch(debug=True, share=True)